In [1]:
import torch

def print_torch_info():
    print("PyTorch version:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("Number of GPUs:", torch.cuda.device_count())
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("No CUDA-enabled GPU found.")

print_torch_info()

PyTorch version: 2.7.1+cu126
CUDA available: True
Number of GPUs: 1
GPU 0: NVIDIA GeForce RTX 2080 Ti


In [4]:
import torch
import torch.nn as nn
from torchvision import transforms, models, datasets

from keras.datasets import mnist
import timm

import numpy as np

In [19]:
def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_resnet_34(device):
    model = models.resnet18(pretrained=True)
    model.to(device)
    model.eval()
    return model

def get_data(batch_size=1):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Lambda(lambda img: img.convert("RGB")),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    data_iter = iter(loader) #Turns the dataloader into batch iterator for data pulling
    return next(data_iter)

def load_vit_base_patch16_224():
    model = timm.create_model('vit_base_patch16_224', pretrained=True)
    return model

def fgsm_attack(image, epsilon, data_gradient):
    sign_gradient = data_gradient.sign()
    perturbed = image + epsilon*sign_gradient
    perturbed = torch.clamp(perturbed, 0, 1) #Guarantees all values are betewen 0 and 1
    return perturbed

def execute_fgsm(model, device, epsilon):
    #Load Data
    images, labels = get_data()
    images, labels = images.to(device), labels.to(device)

    #First Prediction
    outputs = model(images)
    _, init_pred = outputs.max(1)
    print(f"[INFO] Original Prediction: {init_pred.item()}, True Label: {labels.item()}")



if __name__ == "__main__":
    device = get_device()
    print("Using Device: ", device)
    resnet_34_model = load_resnet_34(device)
    epsilon = 0.007
    execute_fgsm(resnet_34_model, device, epsilon)


    
    


Using Device:  cuda
[INFO] Original Prediction: 677, True Label: 1
